In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import os
import shutil

In [2]:
csv.register_dialect('myDialect',delimiter=',', quoting=csv.QUOTE_ALL,skipinitialspace=True)

movie_data = []
genre_list = [
#     'Action',
#     'Adventure',
#     'Animation',
#     'Biography',
    'Comedy',
#     'Crime',
#     'Drama',
#     'Fantasy',
#     'History',
#     'Horror',
#     'Music',
#     'Mystery',
    'Romance',
#     'Sci-Fi',
#     'Sport',
#     'Thriller',
#     'War',
#     'Western'
]
cwd = os.getcwd()
# if os.path.exists(cwd + "/1_movies_per_genre"):
#     shutil.rmtree("1_movies_per_genre", ignore_errors=True)

# os.makedirs("1_movies_per_genre")
num_movies_to_scrape = 100
for idx, genre in enumerate(genre_list):
    i = 0
    j = 0
    page_ctr = 0
    with open(f"1_movies_per_genre/{genre}.csv", "w") as f:
        print(f"Processing {genre}...")
        while(j < num_movies_to_scrape):
            top_50_url = 'https://www.imdb.com/search/title/?title_type=feature&genres={}&sort=num_votes,desc&explore=genres&start={}'.format(genre.lower(), page_ctr*50 + 1)
            page = requests.get(top_50_url, headers = {"Accept-Language": "en-US, en;q=0.5"})
            soup = BeautifulSoup(page.content, 'html.parser')
            table_body = soup.find_all('div', 'lister-item mode-advanced')
            writer = csv.writer(f, dialect='myDialect')
            #write this only for the first page
            if page_ctr == 0:
                writer.writerow(['name', 'year', 'movie_rated', 'run_length', 'genres', 'release_date', 'rating', 'num_raters', 'num_reviews', 'review_url'])
            #Break if movie list is empty
            if not table_body or i>=1000 :
                print(f"{j}/{i} movies scrapped !!!")
                print("50 movies not found in this genre !!!")
                break
            for table_row in table_body:
                i += 1
                rating_url = 'https://www.imdb.com' + table_row.find('div', 'lister-item-content').find('h3', 'lister-item-header').a['href'].split('?')[0]
                review_url = rating_url + 'reviews/_ajax?ref_=undefined&paginationKey='
                page = requests.get(rating_url, headers = {"Accept-Language": "en-US, en;q=0.5"})
                soup = BeautifulSoup(page.content, 'html.parser')
                title_wrapper = soup.find('div', 'title_wrapper')
                name = (str(title_wrapper.h1.contents[0])).split('<span')[0]
                name = name.split('\xa0')[0]
                year = int(title_wrapper.h1.span.a.contents[0])
                rating_value = soup.find('div', 'ratingValue')
                rating_value = str(rating_value.contents[1]).split('"')
                rating = float(rating_value[1].split(' ')[0])
                num_raters = rating_value[1].split('on ')[1]
                num_raters = int(num_raters.split(' ')[0].replace(',', ''))

                subtext = title_wrapper.find('div', 'subtext')
                movie_rated = str(subtext.contents[0]).split('<span')[0]
                movie_rated = movie_rated.replace('\n', ' ')
                movie_rated = movie_rated.strip(' ')
                run_length = subtext.time.contents[0]
                run_length = run_length.replace('\n', ' ')
                run_length = run_length.strip(' ')
                a_href = subtext.find_all('a')
                genres = ""
                release_date = ""
                top_3_genres = ""
                genre_count = 0
                for a in a_href:
                    if str(a.contents[0]) in genre_list:
                        genres = genres + str(a.contents[0]) + "; "
                        if genre_count < 3:
                            top_3_genres = top_3_genres + str(a.contents[0]) + "; "
                            genre_count+=1
                    else:
                        release_date = str(a.contents[0])
                release_date = release_date.replace('\n', ' ')
                release_date = release_date.strip(' ')

                num_reviews = 0
                num_revs = soup.find('div', 'user-comments')
                a_href = num_revs.find_all('a')
                for a in a_href:
                    if (str(a.contents).find('See all my') != -1):
                        continue
                    if (str(a.contents).find('See all ') != -1):
                        num_reviews = str(a.contents).split('See all ')[1]
                        num_reviews = num_reviews.split(' user reviews')[0]
                        num_reviews = int(num_reviews.replace(',', ''))

                if len(movie_rated) == 0:
                    movie_rated = 'Null'
                if len(run_length) == 0:
                    run_length = 'Null'
                if len(genres) == 0:
                    genres = 'Null'
                if len(release_date) == 0:
                    release_date = 'Null'

                movie_data = [name, year, movie_rated, run_length, genres, release_date, rating, num_raters, num_reviews, review_url]
                name = name.replace('/', '_')
                #print (movie_data)
                #If genre is not present in top 3 genres
                if genre in top_3_genres:
#                     if (genre == 'Comedy' and 'Romance' in genres) or (genre == 'Romance' and 'Comedy' in genres):
#                         continue
                    if (int(num_reviews) >= num_movies_to_scrape):
                        j += 1
                        writer.writerow(movie_data)
                if (i%20 == 0):
                    print(f"{j}/{i} movies scrapped !!!")
                    f.flush()
                if j >= 100:
                    print(f"{j}/{i} movies scrapped !!!")
                    print("All movies scrapped !!!")
                    break
            page_ctr += 1
        f.close()

Processing Comedy...
20/20 movies scrapped !!!
40/40 movies scrapped !!!
60/60 movies scrapped !!!
79/80 movies scrapped !!!
99/100 movies scrapped !!!
100/101 movies scrapped !!!
All movies scrapped !!!
Processing Romance...
19/20 movies scrapped !!!
29/40 movies scrapped !!!
44/60 movies scrapped !!!
59/80 movies scrapped !!!
73/100 movies scrapped !!!
85/120 movies scrapped !!!
100/140 movies scrapped !!!
100/140 movies scrapped !!!
All movies scrapped !!!
